# SQL

## Practicum 100 

## Final Project. Part 2

### by Vadim Fridman

#### Step 0. Intro & Table of content

<a class="anchor" id="introduction"></a>

### Table of content 


0. [Step 0: Intro & Table of content](#introduction)


1. [Step 1. Set up a connection and study tables were given](#part1)


2. [Step 2. Study the tables (print the first rows)](#part3)


3. [Step 3. Make an SQL query for each of the tasks.](#part2)
    - [Task1](#part21)
    
    - [Task2](#part22)
    
    - [Task3](#part23)
    
    - [Task4](#part24)  
    
    - [Task5](#part25)
   

4. [Step 4. Overall conclusion](#conclusion)

__Describe the goals of the study.__

Last year coronavirus took the entire world by surprise, changing everyone's daily routine. City dwellers no longer spent their free time outside, going to cafes and malls; more people were at home, reading books. That attracted the attention of startups that rushed to develop new apps for book lovers. 

We are working as a junior Data Analyst. We've been given a SQL database of one of the services competing in this market. It contains data on books, publishers, authors, and customer ratings and reviews of books. 

We are going to use this information to generate a value proposition for a new product.

#### Step 1. Set up a connection and study tables were given
<a class="anchor" id="part1"></a>

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
# pip install psycopg2-binary
# probably you won't need it

In [3]:
db_config = {'user': 'praktikum_student',         # user name
             'pwd': 'Sdf4$2;d-d30pp', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-final-project-db'}          # the name of the data base

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

Here we set up the connection to the database we are going to use. 

Let's have a first look into a tables.

__Step 2. Study the tables (print the first rows)__

<a class="anchor" id="part3"></a>

In [4]:
tables = ['books', 'authors', 'publishers', 'ratings', 'reviews']
for i in tables:
    query = "SELECT * FROM " + i + " LIMIT 5" 
    print("Table name: ", i)
    print("First 5 rows: ")
    print()
    print(pd.io.sql.read_sql(query, con = engine))
    print()
    print()
    
    

Table name:  books
First 5 rows: 

   book_id  author_id                                              title  num_pages publication_date  publisher_id
0        1        546                                       'Salem's Lot        594       2005-11-01            93
1        2        465                 1 000 Places to See Before You Die        992       2003-05-22           336
2        3        407  13 Little Blue Envelopes (Little Blue Envelope...        322       2010-12-21           135
3        4         82  1491: New Revelations of the Americas Before C...        541       2006-10-10           309
4        5        125                                               1776        386       2006-07-04           268


Table name:  authors
First 5 rows: 

   author_id                          author
0          1                      A.S. Byatt
1          2  Aesop/Laura Harris/Laura Gibbs
2          3                 Agatha Christie
3          4                   Alan Brennert
4          

So we have seen how all of the tables we have look like. It will help us to perform the rest of the tasks.

__Step 3. Make an SQL query for each of the tasks.__

<a class="anchor" id="part2"></a>

__1. Find the number of books released after January 1, 2000.__

<a class="anchor" id="part21"></a>

In [17]:
after_2000 = '''SELECT

COUNT( book_id) 

FROM

books

WHERE

publication_date > '2000-01-01' '''

In [18]:
pd.io.sql.read_sql(after_2000, con = engine)

,count
0,819


So the number is  821.  We just counted the number of _*book_id*_'s (assuming that they are unique) in the _*books*_ table.

__2. Find the number of user reviews and the average rating for each book.__
<a class="anchor" id="part22"></a>

In [7]:
# variant with sorting by review with book_title
top_reviews = ''' 

SELECT

books.title, 
AVG(ratings.rating),
COUNT (reviews.review_id) as cnt

FROM

books

INNER JOIN ratings ON ratings.book_id = books.book_id
INNER JOIN reviews ON reviews.book_id = books.book_id

GROUP BY books.book_id

ORDER BY cnt DESC'''


In [8]:
pd.io.sql.read_sql(top_reviews, con = engine)

,title,avg,cnt
0,Twilight (Twilight #1),3.662500,1120
1,The Hobbit or There and Back Again,4.125000,528
2,The Catcher in the Rye,3.825581,516
3,Harry Potter and the Prisoner of Azkaban (Harr...,4.414634,492
4,Harry Potter and the Chamber of Secrets (Harry...,4.287500,480
...,...,...,...
989,Debt of Honor (Jack Ryan #7),3.000000,2
990,Winter Prey (Lucas Davenport #5),4.500000,2
991,Death: The High Cost of Living,3.000000,2
992,The Iliad/The Odyssey,4.000000,2


To find a solution, we had to join _*reviews*_ and _*raitings*_ tables, and calculate desired figures. 

__3. Identify the publisher that has released the greatest number of books with more than 50 pages (this will help you exclude brochures and similar publications from your analysis).__

<a class="anchor" id="part23"></a>

In [9]:
publishers = '''
SELECT

distinct publisher_id,

COUNT (book_id) as cnt

FROM

books 

WHERE 

num_pages > 50

GROUP BY 

publisher_id

ORDER BY 

cnt desc

limit 1;

'''

In [10]:
pd.io.sql.read_sql(publishers, con = engine)

,publisher_id,cnt
0,212,42


We found that the publisher with ID 212 has released the greatest number of books with more than 50 pages (42 books) by filtering out books with 50 pages or less and sorting publishers by a number of books they had published. 

(We were not explicitly asked about the name of the publisher, so we are assuming, that ID is enough. Otherwise we can additionaly request the publishers's name in the corresponding table.)

__4. Identify the author with the highest average book rating (look only at books with at least 50 ratings).__

<a class="anchor" id="part24"></a>

In [11]:
author = ''' 
SELECT 
    authors.author as author_name,
    AVG(ratings.rating) as avg_rating
    
FROM 

authors

INNER JOIN books on books.author_id = authors.author_id

INNER JOIN ratings on ratings.book_id = books.book_id

GROUP BY author_name

HAVING COUNT(ratings.rating) > 50

ORDER BY avg_rating DESC

LIMIT

1
'''

In [12]:
pd.io.sql.read_sql(author, con = engine)

,author_name,avg_rating
0,J.K. Rowling/Mary GrandPré,4.288462


The author with the highest average book rating is J.K. Rowling. 

__5. Find the average number of text reviews among users who rated more than 50 books.__

<a class="anchor" id="part25"></a>

In [13]:
reviews_number = '''

SELECT avg(cnt)


FROM 

    (SELECT

    count(reviews.review_id) as cnt

    FROM

    reviews

    WHERE

    reviews.username in 

    (SELECT 

    ratings.username AS user   

    FROM ratings

    GROUP BY ratings.username

    HAVING COUNT(ratings.rating_id) > 50) 

    GROUP BY

    reviews.username

    ORDER BY

    cnt DESC) as counts'''




In [14]:
pd.io.sql.read_sql(reviews_number, con = engine)

,avg
0,24.333333


The average number of text reviews among users who rated more then 50 books is in output in the previous cell. As we can see, there only 6 users who rated more than 50 books. 

__Step 4. Overall conclusion__

<a class="anchor" id="conclusion"></a>

We found all the requiered information in the database which consist from 5 tables.

We obtained all the information using SQL and used Python only to print out the data in the notebook. 